import libraries

In [1]:
from besos import eppy_funcs as ef
from besos import sampling
from besos.evaluator import EvaluatorEP
from besos.parameters import FieldSelector, FilterSelector, GenericSelector, Parameter
from besos.problem import EPProblem
import pandas as pd

# Building Parameters with Selectors

`Selectors` identify which part of the building model to modify, and how to modify it.  

## Field Selectors
`FieldSelector`s modify individual fields in a building model.  

The example building loaded here contains a `Material` class object named `Mass NonRes Wall Insulation` which has a `Thickness` field.  
We make a selector that modifies this insulation thickness.

In [2]:
FieldSelector(class_name='Material', object_name='Mass NonRes Wall Insulation', field_name='Thickness');

There is only one object in the example building with the name `Mass NonRes Wall Insulation`, so we can ommit the `class_name`.  
The building will be searched for any object with the correct `object_name`.

In [3]:
FieldSelector(object_name='Mass NonRes Wall Insulation', field_name='Thickness');

 If the `class_name` is provided, but the `object_name` is omitted, then the first object with that `class_name` will be used.  
 Since JSON files do not guarentee ordering, this only works for idf files. `field_name` is mandatory.

## Filtering Selectors
`FilterSelectors` allow us to use custom function to select the objects to modify.  
Here we define a function that finds all materials with `Insulation` in their name.  
Then we use this function to modify the thickness of all these materials.

In [4]:
# Load Building
building = ef.get_building(mode='json') # we use the json example file because of how the insulation_filter function works

In [5]:
def insulation_filter(building):
    return [obj for name, obj in building['Material'].items() if 'Insulation' in name]

insulation = FilterSelector(insulation_filter, 'Thickness')

If you have multiple objects of the same type that all share the same field to modify, set `object_name` to `'*'`.

In [6]:
lights = FieldSelector(class_name='Lights', object_name='*', field_name='Watts per Zone Floor Area')

## Parameter scripts using a Generic Selector

Parameters can also be created by defining a function that takes an idf and a value and mutates the idf. These functions can be specific to a certain idf's format, and can perform any arbitrary transformation. Creating these can be more involved.  
`eppy_funcs` contains the functions `one_window` and `wwr_all`. `one_window` removes windows from a building untill it has only one per wall. `wwr_all` takes a building with one window per wall and adjusts it to have a specific window to wall ratio.

BESOS also includes some pre-defined parameter scripts:
+ `wwr(Range)` for window to wall ratio  

Here we define the WWR of all walls in the model to be between 10% and 90%.

In [7]:
window_to_wall = GenericSelector(set=ef.wwr_all, setup=ef.one_window)

# Bundeling the Parameters

We can bundle all the different parameters into a single parameters object.

In [8]:
parameters= [Parameter(selector=x) for x in(insulation, lights, window_to_wall)]

# Extra: Sampling and evaluating the design space
Since `Selectors` do not describe the values they can take on, only where those values go, they are not sufficient to explore the design space.  
We can specify several samples manually to look at the design space.

In [9]:
samples = pd.DataFrame({
    'Thickness': [x/10 for x in range(1,10)]*2,
    'Watts': [8,10,12]*6,
    'wwr': [0.25, 0.5]*9
})
samples

,Thickness,Watts,wwr
0,0.1,8,0.25
1,0.2,10,0.50
2,0.3,12,0.25
3,0.4,8,0.50
4,0.5,10,0.25
5,0.6,12,0.50
6,0.7,8,0.25
7,0.8,10,0.50
8,0.9,12,0.25
9,0.1,8,0.50


We also need to put the `Selectors` in `Parameters` before we can use them in an `Evaluator`.

In [10]:


# the inputs to the problem will be the parameters
# default objective is to minimize the sum of the Electricity:Facility
problem = EPProblem(inputs=parameters)

# The evaluator will take the problem and building file
evaluator = EvaluatorEP(problem, building)

# We can apply some samples to the problem
outputs = evaluator.df_apply(samples ,keep_input=True)


In [11]:
outputs

,Thickness,Watts,wwr,Electricity:Facility
0,0.1,8,0.25,1.605449e+09
1,0.2,10,0.50,1.720154e+09
2,0.3,12,0.25,1.836143e+09
3,0.4,8,0.50,1.544536e+09
4,0.5,10,0.25,1.676760e+09
5,0.6,12,0.50,1.803648e+09
6,0.7,8,0.25,1.518209e+09
7,0.8,10,0.50,1.655640e+09
8,0.9,12,0.25,1.783915e+09
9,0.1,8,0.50,1.605449e+09
